In [20]:
from pyspark.sql import SparkSession

In [21]:
# Aqui a manipulação vai ser feita com SQL

spark = (
    SparkSession
    .builder
    .master("local[*]") # usa nosso processador com todas as threads disponíveis
    .appName('spark_sql')
    .getOrCreate()
)

In [22]:
# dados obtidos em:
# https://dados.gov.br/dados/conjuntos-dados/serie-historica-de-precos-de-combustiveis-e-de-glp?source=post_page-----4185005771e5---------------------------------------
# csv de outubro/2023

df = (
    spark
    .read
    .option('delimiter', ';') # delimitador será ;
    .option('header', 'true') # a primeira linha do csv será o cabeçalho
    .option('inferSchema', 'true') # infere o schema automaticamente (nomes das colunas e tipos de dados)
    .option('encoding', 'ISO-8859-1') # às vezes é dado, às vezes precisamos fazer testes até descobrir
    .csv('./dados/precos-gasolina-etanol-10.csv')
)

In [23]:
df.printSchema()

root
 |-- Regiao - Sigla: string (nullable = true)
 |-- Estado - Sigla: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ da Revenda: string (nullable = true)
 |-- Nome da Rua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- Complemento: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data da Coleta: string (nullable = true)
 |-- Valor de Venda: string (nullable = true)
 |-- Valor de Compra: string (nullable = true)
 |-- Unidade de Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



In [24]:
# pra usar spark SQL precisamos usar esse método abaixo, porque com ele criamos
# uma 'view' temporária (é perdida ao encerrar a sessão atual do spark) como as de 
# bancos de dados, essa 'view' representa o dataframe como tabela e nela podemos usar SQL

df.createOrReplaceTempView('combustiveis') # agora podemos fazer select em 'combustiveis'

In [25]:
spark.sql("""select * from combustiveis""") # mostra os nomes e tipos de dados das colunas

DataFrame[Regiao - Sigla: string, Estado - Sigla: string, Municipio: string, Revenda: string, CNPJ da Revenda: string, Nome da Rua: string, Numero Rua: string, Complemento: string, Bairro: string, Cep: string, Produto: string, Data da Coleta: string, Valor de Venda: string, Valor de Compra: string, Unidade de Medida: string, Bandeira: string]

In [26]:
spark.sql("""select * from combustiveis""").show() # exibe a tabela

+--------------+--------------+---------+--------------------+-------------------+--------------------+----------+--------------------+--------------------+---------+------------------+--------------+--------------+---------------+-----------------+--------+
|Regiao - Sigla|Estado - Sigla|Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|         Complemento|              Bairro|      Cep|           Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|Bandeira|
+--------------+--------------+---------+--------------------+-------------------+--------------------+----------+--------------------+--------------------+---------+------------------+--------------+--------------+---------------+-----------------+--------+
|            NE|            AL|ARAPIRACA|COMERCIAL DE COMB...| 02.817.655/0001-82|RODOVIA AL 220 - ...|       S/N|                NULL|            PLANALTO|57308-000|          GASOLINA|    02/10/2023|          6,08|        

In [27]:
spark.sql("""
    select `Estado - Sigla`, `Produto`, `Valor de Compra`,`Valor de Venda`, `Unidade de Medida` from combustiveis
""").show()

# usamos ` ao invés de ' nos nomes das colunas pois ' faz com que os valores 
# das colunas sejam o próprio nome delas

+--------------+------------------+---------------+--------------+-----------------+
|Estado - Sigla|           Produto|Valor de Compra|Valor de Venda|Unidade de Medida|
+--------------+------------------+---------------+--------------+-----------------+
|            AL|          GASOLINA|           NULL|          6,08|       R$ / litro|
|            AL|GASOLINA ADITIVADA|           NULL|          6,08|       R$ / litro|
|            AL|            ETANOL|           NULL|          4,78|       R$ / litro|
|            AL|          GASOLINA|           NULL|          5,79|       R$ / litro|
|            AL|            ETANOL|           NULL|          4,29|       R$ / litro|
|            AL|          GASOLINA|           NULL|          5,89|       R$ / litro|
|            AL|GASOLINA ADITIVADA|           NULL|          6,09|       R$ / litro|
|            AL|            ETANOL|           NULL|          4,49|       R$ / litro|
|            AL|          GASOLINA|           NULL|          5,89

In [28]:
spark.sql("""
    select * from combustiveis
    where `Valor de Compra` is not null
""").show()

# coluna Valor de Compra é 100% nula

+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+
|Regiao - Sigla|Estado - Sigla|Municipio|Revenda|CNPJ da Revenda|Nome da Rua|Numero Rua|Complemento|Bairro|Cep|Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|Bandeira|
+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+
+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+



In [29]:
# substituindo , por . para converter 'Valor de Venda' pra float
view_precos = spark.sql("""
    select `Estado - Sigla`, `Produto`, regexp_replace(`Valor de Venda`, ",", ".") as `Valor de Venda`, `Unidade de Medida`
    from combustiveis
""")

In [31]:
# a consulta feita com spark.sql retornou um DataFrame, para podermos continuar usando SQL
# é preciso criar a view também pra esse dataframe
view_precos.createOrReplaceTempView('view_precos')
view_precos.show(5)

+--------------+------------------+--------------+-----------------+
|Estado - Sigla|           Produto|Valor de Venda|Unidade de Medida|
+--------------+------------------+--------------+-----------------+
|            AL|          GASOLINA|          6.08|       R$ / litro|
|            AL|GASOLINA ADITIVADA|          6.08|       R$ / litro|
|            AL|            ETANOL|          4.78|       R$ / litro|
|            AL|          GASOLINA|          5.79|       R$ / litro|
|            AL|            ETANOL|          4.29|       R$ / litro|
+--------------+------------------+--------------+-----------------+
only showing top 5 rows



In [41]:
df_diferenca_precos = spark.sql("""
    select
        `Estado - Sigla`, `Produto`, `Unidade de Medida`,
        MIN(`Valor de Venda`) as menor_valor,
        MAX(`Valor de Venda`) as maior_valor,
        MAX(`Valor de Venda`) - MIN(`Valor de Venda`) as diferenca
    from view_precos
    group by all
    order by diferenca desc
""")

In [42]:
df_diferenca_precos.show(10)

+--------------+------------------+-----------------+-----------+-----------+------------------+
|Estado - Sigla|           Produto|Unidade de Medida|menor_valor|maior_valor|         diferenca|
+--------------+------------------+-----------------+-----------+-----------+------------------+
|            SP|GASOLINA ADITIVADA|       R$ / litro|       4.79|       8.69|3.8999999999999995|
|            SP|            ETANOL|       R$ / litro|       2.78|       6.19|3.4100000000000006|
|            SP|          GASOLINA|       R$ / litro|       4.59|       7.59|               3.0|
|            PA|            ETANOL|       R$ / litro|       3.85|        6.6|2.7499999999999996|
|            RS|            ETANOL|       R$ / litro|       3.88|       6.29|              2.41|
|            BA|          GASOLINA|       R$ / litro|       4.69|       6.98|              2.29|
|            SC|            ETANOL|       R$ / litro|       3.84|       5.89|              2.05|
|            AL|            ET